In [ ]:
from pathlib import Path
import pandas as pd
import plotly.express as px

DATA_PATH = Path('../api/data/experiment_logs.csv')
if not DATA_PATH.exists():
    raise FileNotFoundError(f'Arquivo {DATA_PATH} não encontrado. Exporte o CSV de experimentos antes de rodar este notebook.')

df = pd.read_csv(DATA_PATH)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
numeric_cols = ['accuracy', 'bleu', 'rouge_l', 'latency_ms']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

display(df.tail())

In [ ]:
group_cols = ['scenario', 'mode_used']
aggregations = {col: 'mean' for col in ['accuracy', 'bleu', 'rouge_l', 'latency_ms'] if col in df.columns}
aggregations['question'] = 'count'
summary = (
    df.groupby(group_cols).agg(aggregations).rename(columns={'question': 'samples'}).reset_index()
)
display(summary)

if 'accuracy' in summary.columns:
    fig_acc = px.bar(summary, x='mode_used', y='accuracy', color=summary['scenario'].astype(str),
                      barmode='group', title='Accuracy médio por cenário/mode')
    fig_acc.update_xaxes(title_text='Modo')
    fig_acc.update_yaxes(title_text='Accuracy médio')
    fig_acc.show()

if 'latency_ms' in summary.columns:
    fig_latency = px.box(df, x='mode_used', y='latency_ms', color=df['scenario'].astype(str),
                        points='all', title='Distribuição de latência por modo')
    fig_latency.update_yaxes(title_text='Latência (ms)')
    fig_latency.show()